In [1]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,GlobalAveragePooling2D

import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [4]:
import datetime

In [5]:
lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('./HGM-1.0/Below_CAM'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [6]:
# IMG_SIZE = [150,100]

In [7]:
name=['Below_CAM','Front_CAM','Left_CAM', 'Right_CAM']
x_data = []
y_data = []

datacount = 0 # We'll use this to tally how many images are in our dataset
for i in range(0, 4): # Loop over the ten top-level folders
    for j in os.listdir('./HGM-1.0/' + name[i] + '/'):
#         print(j)
        if not j.startswith('.'): # Again avoid hidden folders
            count = 0 # To tally images of a given gesture
            for k in os.listdir('./HGM-1.0/' + 
                                name[i] + '/' + j + '/'):
                                # Loop over the images
                img = Image.open('./HGM-1.0/' + 
                                 name[i] + '/' + j + '/' + k).convert('L')
                                # Read in and convert to greyscale
                img = img.resize((128, 72))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1) # Reshape to be the correct size


In [8]:
print(datacount)

4160


In [9]:
# # check some image
# fig,ax=plt.subplots(5,2)
# fig.set_size_inches(15,15)
# for i in range(5):
#     for j in range (2):
#         l=rn.randint(0,len(y_data))
#         ax[i,j].imshow(x_data[l])
#         ax[i,j].set_title(reverselookup[y_data[l,0]])
        
# plt.tight_layout()

In [10]:
y_data=to_categorical(y_data)

In [11]:
# y_data=to_categorical(y_data)
x_data = x_data.reshape((datacount, 72, 128,1))
x_data = x_data/255
print(x_data.shape)
print(y_data.shape)

(4160, 72, 128, 1)
(4160, 26)


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.2)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.5)

In [13]:
from keras import layers
from keras import models

In [14]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(72, 128,1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(26, activation='softmax'))

In [15]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_validate, y_validate),callbacks=[tensorboard_callback])

Epoch 1/10
52/52 [==============================] - 12s 191ms/step - loss: 3.2665 - accuracy: 0.0374 - val_loss: 3.2606 - val_accuracy: 0.0264
Epoch 2/10
52/52 [==============================] - 6s 120ms/step - loss: 3.2562 - accuracy: 0.0502 - val_loss: 3.2746 - val_accuracy: 0.0529
Epoch 3/10
52/52 [==============================] - 6s 119ms/step - loss: 3.2306 - accuracy: 0.0643 - val_loss: 3.2693 - val_accuracy: 0.0601
Epoch 4/10
52/52 [==============================] - 6s 112ms/step - loss: 3.1791 - accuracy: 0.0760 - val_loss: 3.2204 - val_accuracy: 0.0817
Epoch 5/10
52/52 [==============================] - 6s 117ms/step - loss: 3.1114 - accuracy: 0.1011 - val_loss: 3.2141 - val_accuracy: 0.0817
Epoch 6/10
52/52 [==============================] - 5s 104ms/step - loss: 2.9640 - accuracy: 0.1339 - val_loss: 3.1622 - val_accuracy: 0.0913
Epoch 7/10
52/52 [==============================] - 6s 110ms/step - loss: 2.8124 - accuracy: 0.1693 - val_loss: 3.1032 - val_accuracy: 0.1106
Epoch

In [16]:
%tensorboard --logdir logs/fit

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [17]:
[loss, acc] = model.evaluate(x_test,y_test,verbose=1)
print("Accuracy:" + str(acc))

13/13 [==============================] - 1s 22ms/step - loss: 3.0791 - accuracy: 0.1442
Accuracy:0.14423076808452606


In [18]:
##########Pre-trained model on 20 hand gestures

In [19]:
x_data = []
y_data  = []
datacount = 0 # We'll use this to tally how many images are in our dataset
itemcount= 0

for i in range(0, 4): # Loop over the ten top-level folders
    for j in os.listdir('./HGM-1.0/' + name[i] + '/'):
        itemcount+=1
        if not j.startswith('.'): # Again avoid hidden folders
            count = 0 # To tally images of a given gesture
            for k in os.listdir('./HGM-1.0/' + 
                                name[i] + '/' + j + '/'):
                                # Loop over the images
                img = Image.open('./HGM-1.0/' + 
                                 name[i] + '/' + j + '/' + k).convert('L')
                                # Read in and convert to greyscale
                img = img.resize((128, 72))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
        if itemcount>19:
            break
x_data = np.array(x_data, dtype = 'float32')
y_data = np.array(y_data)
y_data = y_data.reshape(datacount, 1) # Reshape to be the correct size



In [20]:
y_data=to_categorical(y_data)
x_data = x_data.reshape((datacount, 72, 128,1))
x_data = x_data/255

In [21]:
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.2)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.5)

In [22]:
model=models.Sequential()
model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(72, 128,1))) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

In [23]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_validate, y_validate),callbacks=[tensorboard_callback])

Epoch 1/10
12/12 [==============================] - 6s 433ms/step - loss: 2.9754 - accuracy: 0.1375 - val_loss: 2.9366 - val_accuracy: 0.2065
Epoch 2/10
12/12 [==============================] - 2s 129ms/step - loss: 2.9068 - accuracy: 0.1698 - val_loss: 2.7688 - val_accuracy: 0.2065
Epoch 3/10
12/12 [==============================] - 2s 129ms/step - loss: 2.7951 - accuracy: 0.1753 - val_loss: 2.7364 - val_accuracy: 0.2065
Epoch 4/10
12/12 [==============================] - 1s 125ms/step - loss: 2.7400 - accuracy: 0.1796 - val_loss: 2.6610 - val_accuracy: 0.2065
Epoch 5/10
12/12 [==============================] - 1s 122ms/step - loss: 2.6228 - accuracy: 0.1874 - val_loss: 2.7289 - val_accuracy: 0.1630
Epoch 6/10
12/12 [==============================] - 2s 143ms/step - loss: 2.6626 - accuracy: 0.1697 - val_loss: 2.6551 - val_accuracy: 0.1957
Epoch 7/10
12/12 [==============================] - 2s 134ms/step - loss: 2.6550 - accuracy: 0.1781 - val_loss: 2.5808 - val_accuracy: 0.2174
Epoch 

In [24]:
%tensorboard --logdir logs/fit

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [114]:
[loss, acc] = model.evaluate(x_test,y_test,verbose=1)
print("Accuracy:" + str(acc))

3/3 [==============================] - 0s 6ms/step - loss: 2.7459 - accuracy: 0.1304
Accuracy:0.1304347813129425


In [115]:
model.save('gesture_model.h5')

In [116]:
from keras.models import load_model
basic_model = load_model("gesture_model.h5")

In [117]:
for i in range(5):
    basic_model.layers[i].trainable = False

In [118]:
# Freeze the first 3 layers

In [119]:
for i in range(3):
    basic_model.layers[i].trainable = False

In [120]:
for i in range(3,5):
    basic_model.layers[i].trainable = True

In [121]:
# Add three more layers

In [125]:
ll = basic_model.layers[4].output
ll = Dense(32)(ll)
ll = Dense(64)(ll)
ll = Dense(26,activation="softmax")(ll)

new_model =keras.Model(inputs=basic_model.input,outputs=ll)

In [68]:
model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(x_validate, y_validate),callbacks=[tensorboard_callback])

ValueError: You are trying to load a weight file containing 5 layers into a model with 16 layers.

Report:

1.Through the comparison(refer to another notebook file using imagenet), I can see that the final recognition accuracy with the test data is improved dramatcally.

2.The 2 challenges you find and overcome during this assignment: 
One is understanding of transfer learning. I spend a lot of time to learn about the concept of transfer learning;
Another one is how to deal with data shape. You can see from the notebook I didn't cope with this well.